In [ ]:
import os
from dotenv import load_dotenv
from google.adk.agents import Agent, ParallelAgent, SequentialAgent
# 1. Load environment variables from the .env file
# By default, this looks for a .env file in the current directory
load_dotenv()

### LLM lite

In [ ]:
import os, litellm 
from google.adk.models.lite_llm import LiteLlm 
litellm.use_litellm_proxy = True 
litellm_model = LiteLlm( model="gpt-oss-120b")

### Simple agent test

In [ ]:
from datetime import datetime
def get_current_time() -> dict:
    """
    Returns a dictionary with the current time in the format YYYY-MM-DD HH:MM:SS.
    """
    return {"current_time": datetime.now().strftime("%Y-%m-%d %H:%M:%S")}


In [ ]:
root_agent = Agent(
    name="simple_agent",
    model=litellm_model,
    instruction="You are a helpful agent. Usee the following tools to answer the user question: - get_current_time",
    output_key="simple_agent_response",
    tools=[ get_current_time],
)

In [ ]:
from google.adk.runners import InMemoryRunner
from google.adk.plugins.logging_plugin import (
    LoggingPlugin,
)  # <---- 1. Import the Plugin
from google.genai import types
import asyncio

runner = InMemoryRunner(
    agent=root_agent,
    plugins=[
        LoggingPlugin()
    ],  # <---- 2. Add the plugin. Handles standard Observability logging across ALL agents
)
print("✅ Runner configured")

In [ ]:
print("🚀 Running agent with LoggingPlugin...")
print("📊 Watch the comprehensive logging output below:\n")

response = await runner.run_debug("what is the time now")

In [ ]:
model = litellm_model

## Six thinking hats

In [ ]:
# Define the 5 "Thinking" Agents (The Team)
# WHITE HAT: Facts & Data
white_hat = Agent(
    name="WhiteHat",
    model=model,
    instruction="You are the White Hat. Focus ONLY on available data, facts, and information gaps. Be objective and neutral. Do not offer opinions, only verifiable facts or questions about missing data.",
)

# RED HAT: Emotions & Intuition
red_hat = Agent(
    name="RedHat",
    model=model,
    instruction="You are the Red Hat. Focus on intuition, hunches, and emotional reaction. How does this problem make users or the team feel? You do not need to justify your feelings with logic.",
)

# BLACK HAT: Caution & Risk
black_hat = Agent(
    name="BlackHat",
    model=model,
    instruction="You are the Black Hat. Play the devil's advocate. Identify specific risks, potential failure points, downsides, and why this idea might NOT work. Be critical.",
)

# YELLOW HAT: Optimism & Benefits
yellow_hat = Agent(
    name="YellowHat",
    model=model,
    instruction="You are the Yellow Hat. Focus on the positives. What are the benefits? What is the best-case scenario? Why will this work?",
)

# GREEN HAT: Creativity & Alternatives
green_hat = Agent(
    name="GreenHat",
    model=model,
    instruction="You are the Green Hat. Focus on creativity, new ideas, and alternatives. How can we bypass constraints? What are some wild or out-of-the-box solutions?",
)



In [ ]:
# Create the Parallel Group
# This runs all 5 agents at the same time on the user prompt
thinking_team = ParallelAgent(
    name="SixHatsBrainstorm",
    sub_agents=[white_hat, red_hat, black_hat, yellow_hat, green_hat]
)

# 4. Define the Blue Hat (The Manager)
# This agent sees the combined output of the team and makes the plan
blue_hat = Agent(
    name="BlueHat",
    model=model,
    instruction="""
    You are the Blue Hat. You are the manager of the thinking process. 
    You will receive input from 5 other agents (White, Red, Black, Yellow, Green).
    Your job is to:
    1. Synthesize their disparate points of view.
    2. Resolve conflicts between the Black (Risk) and Yellow (Optimism) hats.
    3. Produce a final, well-rounded decision or action plan.
    """,
)



In [ ]:
# Create the Final Workflow
# First run the team (Parallel), then run the manager (Sequential)
solver_workflow = SequentialAgent(
    name="SixHatsSolver",
    sub_agents=[thinking_team, blue_hat]
)



In [ ]:
# --- EXECUTION ---
# user_input = "Should we switch our backend database from PostgreSQL to a NoSQL solution for our startup?"
# result = solver_workflow.run(user_input)
# print(result)

In [ ]:
from google.adk.runners import InMemoryRunner
from google.adk.plugins.logging_plugin import (
    LoggingPlugin,
)  # <---- 1. Import the Plugin
from google.genai import types
import asyncio

runner = InMemoryRunner(
    agent=solver_workflow,
    plugins=[
        LoggingPlugin()
    ],  # <---- 2. Add the plugin. Handles standard Observability logging across ALL agents
)
print("✅ Runner configured")

In [ ]:
print("🚀 Running agent with LoggingPlugin...")
print("📊 Watch the comprehensive logging output below:\n")

response = await runner.run_debug("Did the chicken come first of the egg?")